> NOMBRE: Elizon Frank Carcausto Mamani

> CODIGO: 170427

# Instalacion e importacion de recursos

In [2]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 56.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=41092b896263646c945966c9476623c1bb965a642dc21da0a398bcc8c44284a0
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [36]:
import math
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

# 1. Similitud de Cosenos

In [44]:
# "x" y "y" son dos array RDD numericos 
def simili_cos(x,y):
  def product_p(x,y):
    # reduce: sumar los productos
    # map: multiplicar valores de x[0] y x [1] de la tupla x
    return x.zip(y).map(lambda x: x[0]*x[1]).reduce(lambda x , y :x+y) 
  return product_p(x,y)/(product_p(x,x)*product_p(y,y))

sc =SparkContext.getOrCreate()
x = sc.parallelize(range(0,25),5)
y = sc.parallelize(range(50, 75),5)
print(simili_cos(x,y))

4.1696349997904704e-05


# 2. Estandarizacion

In [46]:
# "x" es un array RDD numerico
def estandarizacion(x):
  # calculamos la media y desviacion estandar del RDD
  media, std  = x.mean(), x.stdev()
  # Mapeamos cada valor de la lista para estandarizar
  lista= x.map(lambda xi : (xi-media)/std)
  return lista

Vec = sc.parallelize(range(25,50),5)
VectorEs=estandarizacion(Vec)
print(VectorEs.collect())

[-1.6641005886756874, -1.52542553961938, -1.386750490563073, -1.2480754415067656, -1.1094003924504583, -0.9707253433941511, -0.8320502943378437, -0.6933752452815365, -0.5547001962252291, -0.41602514716892186, -0.2773500981126146, -0.1386750490563073, 0.0, 0.1386750490563073, 0.2773500981126146, 0.41602514716892186, 0.5547001962252291, 0.6933752452815365, 0.8320502943378437, 0.9707253433941511, 1.1094003924504583, 1.2480754415067656, 1.386750490563073, 1.52542553961938, 1.6641005886756874]


# 3. Escalonamiento

Creamos un data set para los dos siguiente ejemplos

In [45]:
from pyspark.ml.linalg import Vectors

# Creamos un dataset de prueba
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])

In [39]:
from pyspark.ml.feature import MinMaxScaler

# Configuramos el MinMaxScaler (importada de una libreria)
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# ajustamos el scaler para calcular las estadísticas resumidas
scalerModel = scaler.fit(dataFrame)

# Cambiar la escala de cada característica al rango [mínimo, máximo]
scaledData = scalerModel.transform(dataFrame)
print("Funciones escaladas al rango: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
scaledData.select("features", "scaledFeatures").show()

Funciones escaladas al rango: [0.000000, 1.000000]
+--------------+--------------+
|      features|scaledFeatures|
+--------------+--------------+
|[1.0,0.1,-1.0]|     (3,[],[])|
| [2.0,1.1,1.0]| [0.5,0.1,0.5]|
|[3.0,10.1,3.0]| [1.0,1.0,1.0]|
+--------------+--------------+



# 4. Normalizacion



In [40]:
from pyspark.ml.feature import Normalizer

# Normaliza cada vector usando la Normalizer.
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
l1NormData = normalizer.transform(dataFrame)
print("Normalizando....")
l1NormData.show()

Normalizando....
+---+--------------+--------------------+
| id|      features|        normFeatures|
+---+--------------+--------------------+
|  0|[1.0,0.1,-1.0]|[0.47619047619047...|
|  1| [2.0,1.1,1.0]|[0.48780487804878...|
|  2|[3.0,10.1,3.0]|[0.18633540372670...|
+---+--------------+--------------------+



# 5. Bag of Words

In [43]:
# Importamos la librerías a utilizar
from pyspark.sql.functions import explode, split, regexp_replace, col, lower
# Data
data = [
    ['A', 'Jan', 'John', 'don quijote salio de la mancha'],
    ['B', 'Feb', 'Mary', 'mancha de don quijote no lo deja en paz'],
    ['C', 'Mar', 'Luke', 'sancho su fiel escudero'],
    ['D', 'Apr', 'Mark', 'quijote y sancho, el cabalo y el burro']
]
columns = ['Title', 'Month', 'Author', 'Document']
df = spark.createDataFrame(data, columns)
#Mostrando el df
print(" Data set")
df.show()

#BOG
print(" Bag of Word")
df.select(explode(split(regexp_replace("Document", "[,.-]", " "), "\s+")).alias("word"))\
    .groupby(lower(col("word")).alias("lower"))\
    .count()\
    .show()

 Data set
+-----+-----+------+--------------------+
|Title|Month|Author|            Document|
+-----+-----+------+--------------------+
|    A|  Jan|  John|don quijote salio...|
|    B|  Feb|  Mary|mancha de don qui...|
|    C|  Mar|  Luke|sancho su fiel es...|
|    D|  Apr|  Mark|quijote y sancho,...|
+-----+-----+------+--------------------+

 Bag of Word
+--------+-----+
|   lower|count|
+--------+-----+
|      en|    1|
|     don|    2|
|  cabalo|    1|
|  sancho|    2|
|    fiel|    1|
|  mancha|    2|
|      de|    2|
|      el|    2|
|      su|    1|
| quijote|    3|
|       y|    2|
|escudero|    1|
|    deja|    1|
|      no|    1|
|   salio|    1|
|   burro|    1|
|      la|    1|
|      lo|    1|
|     paz|    1|
+--------+-----+

